In [ ]:
#Import database setup
import setup_env

In [ ]:
import statsmodels.api as sm

In [ ]:
# SARIMA example
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.ar_model import ARResults
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
# grid search sarima hyperparameters
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [ ]:
engine = setup_env.get_database()
print(engine)
try:
    con = engine.raw_connection()
    con.cursor().execute("SET SCHEMA '{}'".format("here_traffic"))
except:
    print("Con: DB Verbindung prüfen!") 
    exit

In [ ]:
rows = "*"
sel_link_id = 563933733
sel_min_confidence = 0
sel_max_weekday = 5
sel_func_classes = ('4','3')

In [ ]:
sql_query = f"""
    SELECT DISTINCT tr.*  
    FROM here_traffic.stuttgart_traffic tr
    JOIN here_streets.fc_streets_all_2018q3 st on tr.link_id = st.link_id
    WHERE tr.link_id = {sel_link_id}
    AND tr.confidence > {sel_min_confidence}
    AND tr.weekday_n < {sel_max_weekday}
    --AND st.func_class in {sel_func_classes}
    AND tr.dir_travel = 'F'
    LIMIT 100000
"""

In [ ]:
%%time
pd_read = pd.read_sql_query(sql_query, con)
df.shape

In [ ]:
df = pd.DataFrame(
    pd_read,
    columns=[
        "id_pk",        #0
        "link_id",      #1
        "mean_kmh",     #2
        "min_kmh",      #3
        "max_kmh",      #4
        "datum_zeit",   #5
        "weekday_n",    #6
        "epoch_60",     #7
        "freeflow_kmh",
        "confidence",
        "count_n"
    ],
)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.shape
df.info

In [ ]:
plot_acf(df['count_n'], lags=72)

plot_pacf(df['count_n'], lags=72)

plt.show()

In [ ]:
df_diff = df['count_n'].diff(periods=1)
df_diff.dropna(inplace=True)
df_diff

In [ ]:
plot_acf(df_diff, lags=72)

plot_pacf(df_diff, lags=72)

plt.show()

In [ ]:
%matplotlib inline
count_plt = sns.lineplot(data=df_diff[:144], legend=False)
sns.set()
plt.figure(figsize=(60,10))
plt.savefig('imgs/lag_24.png')
plt.show()


In [ ]:
df.set_index('datum_zeit', inplace=True, drop=True)
df.sort_index(inplace=True)

In [ ]:
df4 = df[(df.index  > '2018-05-01 00:00:00') & (df.index < '2018-09-1 00:00:00')]

In [ ]:
pd.date_range(start = '2018-05-01 00:00:00', end = '2018-09-1 00:00:00' ).difference(df4.index)

In [ ]:
df4.index.to_timestamp()
df4

In [ ]:
from sklearn.metrics import mean_squared_error
series = df['count_n']
# split dataset
X = series.values
print(X)
train, test = X[1:len(X)-48], X[len(X)-48:]
# train autoregression
model = SARIMAX(train)
model_fit = model.fit()
window = model_fit.k_ar
coef = model_fit.params
# walk forward over time steps in test
history = train[len(train)-window:]
history = [history[i] for i in range(len(history))]
predictions = list()
for t in range(len(test)):
	length = len(history)
	lag = [history[i] for i in range(length-window,length)]
	yhat = coef[0]
	for d in range(window):
		yhat += coef[d+1] * lag[window-d-1]
	obs = test[t]
	predictions.append(yhat)
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
series = df['count_n']
# split dataset
X = series.values
print(X)
train, test = X[1:len(X)-72], X[len(X)-72:]
# train autoregression
model = AR(train)
model_fit = model.fit()
window = model_fit.k_ar
coef = model_fit.params
# walk forward over time steps in test
history = train[len(train)-window:]
history = [history[i] for i in range(len(history))]
predictions = list()
for t in range(len(test)):
	length = len(history)
	lag = [history[i] for i in range(length-window,length)]
	yhat = coef[0]
	for d in range(window):
		yhat += coef[d+1] * lag[window-d-1]
	obs = test[t]
	predictions.append(yhat)
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()

In [ ]:
mod = sm.tsa.statespace.SARIMAX(df4['count_n'], trend='n',  order=(1,0,0), seasonal_order=(1,1,0,24), simple_differencing=True)

In [ ]:
import warnings
from pandas import read_csv
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit(disp=0)
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	error = mean_squared_error(test, predictions)
	return error

# Evaluation von ARIMA params
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_cfg = mse, order
					print('Model%s MSE=%.3f' % (order,mse))
				except:
					continue
	print('Min ARIMA%s MSE=%.3f' % (best_cfg, best_score))

series = df4['count_n']
# evaluate parameters
p_values = [0, 1, 24]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values)